### Q1. Continuing the Vector

Our Vector implementation so far looks like this

In [1]:
#example from Fluent
from array import array 
import reprlib
import math
import numbers
import functools
import operator
import itertools

class Vector:
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components) 
        components = components[components.find('['):-1] 
        return 'Vector({})'.format(components)
    
    def __eq__(self, other):
        if isinstance(other, Vector):
            return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))
        else:
            return NotImplemented
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self): 
        return bool(abs(self))
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index): 
        cls = type(self)
        
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral): 
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers' 
            raise TypeError(msg.format(cls))
            
    def __neg__(self):
        return Vector(-x for x in self) 
    
    def __pos__(self):
        return Vector(self)
    
    def _check_length_helper(self , rhs):
        if not len(self)==len(rhs):
            raise ValueError(str(self)+' and '+str(rhs)+' must have the same length')
    
    def __add__(self, rhs):
        try:
            if isinstance(rhs, numbers.Real):
                return Vector(a + rhs for a in self) 
            else: #
                self._check_length_helper(rhs)
                pairs = zip(self, rhs)
                return Vector(a + b for a, b in pairs)
        except TypeError:
            raise NotImplemented
    
    def __radd__(self, other): # other + self delegates to __add__
        return self + other

First, implement all the ad-hoc tests so far as doctests. Then implement multiplication and subtraction for this class. Notice that this is currently a non-mutable vector.

In [69]:
#your code here
# https://docs.python.org/3/reference/datamodel.html#object.__sub__
class Vector:
    """
    >>> A=Vector([1,2,3,1])
    >>> B=Vector([1,2,3,1])
    >>> A==B
    True
    >>> abs(A)
    3.872983346207417
    >>> bool(A)
    True
    >>> C = Vector([])
    >>> bool(C)
    False
    >>> len(A)
    4
    >>> A[3]
    1.0
    >>> -A
    Vector([-1.0, -2.0, -3.0, -1.0])
    >>> A + B
    Vector([2.0, 4.0, 6.0, 2.0])
    >>> 10 + B
    Vector([11.0, 12.0, 13.0, 11.0])
    >>> B + 10
    Vector([11.0, 12.0, 13.0, 11.0])
    >>> D = Vector([1,2])
    >>> A + D
    Traceback (most recent call last):
        ...
    ValueError: Vector([1.0, 2.0, 3.0, 1.0]) and Vector([1.0, 2.0]) must have the same length
    >>> A - 3
    Vector([-2.0, -1.0, 0.0, -2.0])
    >>> 3 - A
    Vector([2.0, 1.0, 0.0, 2.0])
    >>> A - B
    Vector([0.0, 0.0, 0.0, 0.0])
    >>> A*B
    Vector([1.0, 4.0, 9.0, 1.0])
    >>> A*3
    Vector([3.0, 6.0, 9.0, 3.0])
    >>> 4*B
    Vector([4.0, 8.0, 12.0, 4.0])
    """
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components) 
        components = components[components.find('['):-1] 
        return 'Vector({})'.format(components)
    
    def __eq__(self, other):
        if isinstance(other, Vector):
            return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))
        else:
            return NotImplemented
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self): 
        return bool(abs(self))
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index): 
        cls = type(self)
        
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral): 
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers' 
            raise TypeError(msg.format(cls))
            
    def __neg__(self):
        return Vector(-x for x in self) 
    
    def __pos__(self):
        return Vector(self)
    
    def _check_length_helper(self , rhs):
        if not len(self)==len(rhs):
            raise ValueError(str(self)+' and '+str(rhs)+' must have the same length')
    
    def __add__(self, rhs):
        try:
            # for scalar
            if isinstance(rhs, numbers.Real):
                return Vector(a + rhs for a in self) 
            # for vector
            else: #
                self._check_length_helper(rhs)
                pairs = zip(self, rhs)
                return Vector(a + b for a, b in pairs)
        except TypeError:
            raise NotImplemented
    
    # Python will look for __add__ on lefthand object, if none, then will look for radd on the right hand object
    # this switches the order, takes self (Which in this case is on the rhs, and then adds it to toher)
    def __radd__(self, other): # other + self delegates to __add__
        return self + other

    def __sub__(self, rhs): 
        """
        very similar to addition
        """
        try:
            # for scalar
            if isinstance(rhs, numbers.Real):
                # uses __radd__ already implemented for self
                return -rhs + self
            # for vector
            else: #
                self._check_length_helper(rhs)
                pairs = zip(self, rhs)
                return Vector(a - b for a, b in pairs)
        except TypeError:
            raise NotImplemented

    def __rsub__(self, other): # other + self delegates to __add__
        # convert other to negative then add
        # can do this because __neg__ already implemented
        return -self + other
            
    def __mul__(self, rhs):
        """
        vectorized multiplication like numpy, more complicated
        """
        try: 
            if isinstance(rhs, numbers.Real):
                # uses __radd__ already implemented for self
                return Vector(a*rhs for a in self)
            else: 
                self._check_length_helper(rhs)
                pairs = zip(self, rhs)
                return Vector(a*b for a, b in pairs)
        except TypeError: 
            raise NotImplemented

    def __rmul__(self, lhs):
        """
        vectorized multiplication like numpy, more complicated
        """
        return self*lhs
            

In [70]:
# my code here
from doctest import run_docstring_examples as dtest
dtest(Vector, globals(), verbose=True)

Finding tests in NoName
Trying:
    A=Vector([1,2,3,1])
Expecting nothing
ok
Trying:
    B=Vector([1,2,3,1])
Expecting nothing
ok
Trying:
    A==B
Expecting:
    True
ok
Trying:
    abs(A)
Expecting:
    3.872983346207417
ok
Trying:
    bool(A)
Expecting:
    True
ok
Trying:
    C = Vector([])
Expecting nothing
ok
Trying:
    bool(C)
Expecting:
    False
ok
Trying:
    len(A)
Expecting:
    4
ok
Trying:
    A[3]
Expecting:
    1.0
ok
Trying:
    -A
Expecting:
    Vector([-1.0, -2.0, -3.0, -1.0])
ok
Trying:
    A + B
Expecting:
    Vector([2.0, 4.0, 6.0, 2.0])
ok
Trying:
    10 + B
Expecting:
    Vector([11.0, 12.0, 13.0, 11.0])
ok
Trying:
    B + 10
Expecting:
    Vector([11.0, 12.0, 13.0, 11.0])
ok
Trying:
    D = Vector([1,2])
Expecting nothing
ok
Trying:
    A + D
Expecting:
    Traceback (most recent call last):
        ...
    ValueError: Vector([1.0, 2.0, 3.0, 1.0]) and Vector([1.0, 2.0]) must have the same length
ok
Trying:
    A - 3
Expecting:
    Vector([-2.0, -1.0, 0.0, -2.0]

In [72]:
v1 = Vector([1,2])
v2=Vector([3,4])
v3=Vector([1,2,3])
v1+1,v1+[1,2], 1 + v1, [1,2] + v1, v1 + v2, v2 + v1

(Vector([2.0, 3.0]),
 Vector([2.0, 4.0]),
 Vector([2.0, 3.0]),
 Vector([2.0, 4.0]),
 Vector([4.0, 6.0]),
 Vector([4.0, 6.0]))

In [73]:
v1 * v2, v2* v1, 4*v1, v1*4.9

(Vector([3.0, 8.0]),
 Vector([3.0, 8.0]),
 Vector([4.0, 8.0]),
 Vector([4.9, 9.8]))

### Q2. Mixins for functionality

Here is a set of methods that logs dictionary access

In [75]:
# this is multiple inheritance
class LoggedMappingMixin: 
    '''
    Add logging to get/set/delete operations for debugging. 
    '''
    __slots__ = ()
    def __getitem__(self, key): 
        print('Getting ' + str(key)) 
        return super().__getitem__(key)
    def __setitem__(self, key, value): 
        print('Setting {} = {!r}'.format(key, value)) 
        return super().__setitem__(key, value)
    def __delitem__(self, key): 
        print('Deleting ' + str(key)) 
        return super().__delitem__(key)

Notice the use of `super()` here. `super()` is the same as `super(self.__class__, self)`. But we dont have a parent!

What is going on? You tell me the answer to this when you inherit a `LoggedDict` with no implementation from both `LoggedMappingMixin` and `dict`. Which order must you inherit in? Play with the `mro` method and figure this out.

In [76]:
# your code here

# this is multiple inheritance
# this is putting LoggedMappingMixin in between LoggedDict and dict in the method resolution order (mro)
# doesn't affect LoggedMappingMixin
# as if inherited first from LoggedMappingMixin and making LoggedMappingMixin inherit from dict
# inheriting first from LoggedMappingMixin, and then dict
class LoggedDict(LoggedMappingMixin, dict): 
    """
    """
    pass

In [77]:
#your code here
# when method is called, it will look in order. First will look in LoggedDict, finds nothing. 
# Then looks in LoggedMappingMixin, which then calls super to dict
LoggedDict.mro()

[__main__.LoggedDict, __main__.LoggedMappingMixin, dict, object]

*your answer here*

This is multiple inheritance. This is putting LoggedMappingMixin in between LoggedDict and dict in the method resolution order (mro). By creating a LoggedDict class that first inherits from LoggedMappingMixin and then dict, when LoggedDict is called, it will look in the order output by the `mro` method. First it will look in LoggedDict, where it finds nothing, then it will look in LoggedMappingMixin, which then calls super onto the inserted parent, dict.


### Q3. The Pavlos Problem

ABC's and doctests. The Pavlos problem.

Introspection of a class hierarchy is helped by:
`__subclasses__()` and `_abc_registry` which give us concrete subclasses and virtual subclasses respectively. We can use this to fully document an interface via an example.

In [79]:
import abc

# abc.ABC means you can register implementations. An abc.ABC is interface that you can then register implementations 
# of
class StackInterface(abc.ABC):
    """
    >>> a = Stack()
    >>> a.push(1)
    >>> a.push(2)
    >>> a.peek()
    2
    >>> a.pop()
    2
    >>> a.pop()
    1
    >>> a.peek()
    >>> a.pop()
    """
    
    @abc.abstractmethod
    def push(self, value):
        "Push value onto the stack. Return None"
        
    @abc.abstractmethod
    def pop(self):
        "Pop value from Stack. Return None if nothingon stack"
        
    @abc.abstractmethod
    def peek(self):
        "Peeak at top of stack. Return None if empty"

Implement `ListStack` using a python list

In [99]:
#your code here

# write stack with python list is trivial
# you can register it or inherit it, but either way should be able to test 
# use the hint __subclasses__() and _abc_registry w
# use tests -- globals() is the global where this works
# need to enhance globals() to make this works

# pass into Doctests some globals
# use this environment in the test
# implement CapitalStack

# register it
class ListStack:

    def __init__(self):
        self.items = []
    
    def push(self, item):
        self.items.append(item)
    
    def pop(self):
        return self.items.pop()
    
    def peek(self):
        print(self.items[len(self.items)-1])
    
StackInterface.register(ListStack)

__main__.ListStack

In [100]:
# won't be in subclasses because it's a virtual subclass that was registered not directly inherited from
StackInterface.__subclasses__()

[]

In [103]:
list(StackInterface._abc_registry)

[__main__.ListStack,
 __main__.ListStack,
 __main__.ListStack,
 __main__.ListStack,
 __main__.ListStack]

In [102]:
#your code here
# https://docs.python.org/3/library/doctest.html#which-docstrings-are-examine

# Benefit: This allows you to write one set of tests on an interface and check that every possible implementation conforms to
# that interface

test_globals = dict(globals())
# can iterate through StackInterface.__subclasses__() or StackInterface._abc_registry_()
# can do this with each implementation instead of liststack
test_globals['Stack'] = ListStack
dtest(StackInterface, test_globals, verbose=True)

Finding tests in NoName
Trying:
    a = Stack()
Expecting nothing
ok
Trying:
    a.push(1)
Expecting nothing
ok
Trying:
    a.push(2)
Expecting nothing
ok
Trying:
    a.peek()
Expecting:
    2
ok
Trying:
    a.pop()
Expecting:
    2
ok
Trying:
    a.pop()
Expecting:
    1
ok
Trying:
    a.peek()
Expecting nothing
**********************************************************************
File "__main__", line ?, in NoName
Failed example:
    a.peek()
Exception raised:
    Traceback (most recent call last):
      File "/Users/amylee/anaconda/envs/ana24py35/lib/python3.5/doctest.py", line 1320, in __run
        compileflags, 1), test.globs)
      File "<doctest NoName[6]>", line 1, in <module>
        a.peek()
      File "<ipython-input-99-211bed22f2d1>", line 26, in peek
        print(self.items[len(self.items)-1])
    IndexError: list index out of range
Trying:
    a.pop()
Expecting nothing
**********************************************************************
File "__main__", line ?, in No

How do we test this using the tests in `StackInterface`? And in general for other virtual or real subclasses? Show this here. (work out doing this from a file at home, you dont need to answer the file case here). This recipe wont work with py.test

### Q4. Your Timeseries Project (GROUP)

Operator overloading on your `TimeSeries` class.

Your `TimeSeries` class should be, by now, a well documented, well tested, mutable, class which implements:

- `__getitem__`: to get a value for a given time
- `__setitem__`: set the value for the given time
- `__contains__`: is a value in the values
- `__iter__`: iterates over values. (This might have iterated over tuples of (time, value) pairs earlier
- `values`: returns a numpy array of values
- `itervalues`: returns an iterator over them
- `times`: returns a numpy array of times
- `itertimes`: returns an iterator over them
- `items`: returns a list of time-value tuple pairs
- `iteritems`: returns an iterator over these
- `__len__`: returns a length.
- `__repr__`: abbreviating spring representation

Add to these methods(again well tested):

- infix addition, subtraction, equality and multiplication. Here you must check that the lengths are equal and that the time domains are the same for the case of the operations on a TimeSeries (the latter implies the former). Return a `ValueError` in case this fails:

`ValueError(str(self)+' and '+str(rhs)+' must have the same time points')`

Let these be elementwise operations, as we might expect from a numpy array-like thing. As before, handle the case of a constant.
- unary `__abs__`, `__bool__`, `__neg__`, and `__pos__` with the same semantics as the `Vector` class above.


A question that might arise is what to do if we add numpy arrays or regular python lists. These should fail with `raise NotImplemented` as we dont have time associated. An option might have been to associate the array with the time indexing of the other array, but this is making too many assumptions: the user can do this explicitly.

You will probably have to catch another exception for this to happen.

Put this code into your project repo.

In [ ]:
# make sure these things are all added already
# then add these other things
# make sure the times are the same between the different times.. better have same indexces for the values inside of the time series
# support the element-wise addition
# implementing this for two time series ... handle case of constant. Once have template. 
# in the near future, we are going to reflect it into the language we've started defining

### TO READ: Numpy ufuncs and function overloading

Check this out. Read http://docs.scipy.org/doc/numpy-dev/reference/arrays.classes.html#special-attributes-and-methods to understand how this works. We will use it later.

In [1]:
import numpy as np
import pandas as pd
p=pd.Series([1,2,3])
print(type(p))
p2=np.exp(p)
p2, type(p2)